# Functions

In [34]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

In [35]:
# Functions for binning (discreetising) numeric fields

def distance_binning(dataset, field, no_of_bins):
    
    # Buckets numerical field data into equally sized bucket thresholds
    # Returns updated dataframe
    
    # Inputs: 
    #    dataset:     Pandas dataframe containing dataset 
    #    field:       Field to be binned
    #    no_of_bins:  Required number of bins
    
    bins = np.linspace(dataset[field].min(),dataset[field].max(),no_of_bins+1)
    labels = [i for i in range(1,no_of_bins+1)]
    dataset_new = dataset.copy()
    dataset_new[field] = pd.cut(dataset_new[field], bins=bins, labels=labels, include_lowest=True)
    return dataset_new

def frequency_binning(dataset, field, no_of_bins):
    
    # Buckets numerical field data into buckets containing roughly equal number of records
    # Returns updated dataframe
    
    # Inputs: 
    #    dataset:     Pandas dataframe containing dataset 
    #    field:       Field to be binned
    #    no_of_bins:  Required number of bins
    
    labels = [i for i in range(1,no_of_bins+1)]
    dataset_new = dataset.copy()
    dataset_new[field] = pd.qcut(dataset_new[field], q=no_of_bins, labels=labels)
    return dataset_new


# Normalises the dataset
# you have to remove all non-numerical columns before you call this function
# data : the dataset you want to normalise 
# omit : colums to omit from the normalisation (as a 1d list)
def NormaliseDS(data, omit=[]):
    if (len(omit)>0):
        temp = data[omit]
        data = data.drop(columns=omit, axis=1)
    column = list(data.columns)
    transform = MinMaxScaler().fit_transform(data)
    newDF = pd.DataFrame(transform, columns=column)
    if (len(omit)>0):
        dataF = pd.concat([newDF, temp], axis=1)
        return(dataF)
    else:
        return(newDF)

# standardises the dataset
# you have to remove all non-numerical columns before you call this function
# data : the dataset you want to standardise
def StandardiseDS(data, omit=[]):
    if (len(omit)>0):
        temp = data[omit]
        data = data.drop(columns=omit, axis=1)
    column = list(data.columns)
    strdDS = StandardScaler().fit_transform(data)
    strdDF = pd.DataFrame(strdDS, columns=column)
    if (len(omit)>0):
        dataF = pd.concat([strdDF, temp], axis=1)
        return(dataF)
    else:
        return(strdDF)

# renames the column names in a dataset so it is one word instead of a space
# data : the dataset you want to rename 
# cols : a dictionary of the current columns you want to rename and the name you want to rename it to (new name should have no spaces)
def RenameFields(data, cols):
    data2 = data.rename(columns=cols)
    return(data2)

# this fucntion divides the possible results of the dataset (ie the quality of a wine) and up-samples the dataset so the data is balanced
# dataset (dataframe) : the dataset you want to up-sample
# targetClass (string) : the name of the column that has the results
# startPoint (int) : the lowest class by number in the dataset(for wine lowest quality is 3, Highest quality is 9 )
# biggestClass (int) : the class that has the highest number of rows
def OverSample (dataset, targetClass, startPoint, biggestClass):
    temp = []
    temp2 = []
    loopRange = (startPoint + len(dataset[targetClass].value_counts()))
    for i in range(startPoint, loopRange):
        temp.append(dataset[dataset[targetClass] == i])
    for j in temp:
        temp2.append(j.sample(len(temp[(biggestClass-startPoint)]), replace=True))
        
    final = pd.concat(temp2)
    return(final)


# Pre-Processing

In [36]:
# Loads the datasets
wine = pd.read_csv('winequalityN.csv')
mushrooms = pd.read_csv('mushrooms.csv')

# renames the columns
dsCols = {'fixed acidity': 'fixed-acidity', 'volatile acidity': 'volatile-acidity', 'citric acid': 'citric-acid', 'residual sugar':'residual-sugar', 'free sulfur dioxide':'free-sulfur-dioxide', 'total sulfur dioxide':'total-sulfur-dioxide'}
wine = RenameFields(wine, dsCols)

################ NULL Checks ################
print(mushrooms.isnull().sum())
print(wine.isnull().sum())
# removes rows with null values
wine = wine.dropna()


mushroomsTemp = mushrooms['class'].replace(['p', 'e'],[1, 0])
mushrooms.drop('class', inplace=True, axis=1)
mushrooms = pd.concat([mushrooms, mushroomsTemp], axis=1)
print(mushrooms)

################ Normalisation and Standardisation ################

# temp removes the worded fields
wineTemp = wine['type'].replace(['white', 'red'],[1, 0])   # binary encodes white and red wine types (white=1, red=0)
wine.drop('type', inplace=True, axis=1)
wine = pd.concat([wine, wineTemp], axis=1)
print(wine)

#normalises the dataset (omit is used to remove colums that do not need to be normalised ie bool, one hot, etc)
normalwine = NormaliseDS(wine, omit=['quality', 'type'])
#standardises the dataset (omit is used to remove colums that do not need to be normalised ie bool, one hot, etc)
standardwine = StandardiseDS(normalwine, omit=['quality', 'type'])
print("standard")
print(standardwine)


################ Balancing ################                COULD ALSO TRY BUCKTING TO BALANCE THE DATA

# prints the distribution of wine quality (quality is between 3 and 9, has most of the wines inbetween )
print(standardwine.quality.value_counts())      # Doing this we can see that the lowest quality is 3 (so that is the startPoint), and the class with the most rows is 6 (so 6 is the biggestClass)

balancedDataWine = OverSample(standardwine, 'quality', startPoint=3, biggestClass=6)
print(balancedDataWine.quality.value_counts())     # we can now see that the results are up Sampled to match the biggest dataset

# balancing for mushroom dataset (MUSHROOMS IS ALREADY KINDA BALANCED)
print(mushrooms['class'].value_counts())      # dataset is already balanced enough
 


######## OPTIONAL Wine Binning ########
#wine_data = pd.read_csv('winequalityN.csv')

#wine_data_binned = distance_binning(wine_data, 'pH', 5)
#print("Value counts")
#print(wine_data_binned['pH'].value_counts())
#print("\nBucket Min Max Size")
#for i in wine_data_binned['pH'].unique():
#    max = wine_data[wine_data_binned['pH']==i]['pH'].max()
#    min = wine_data[wine_data_binned['pH']==i]['pH'].min()  
#    print(str(i) + " " + str(min) + " " + str(max) + " " + str(round(max-min,2)))
    
    
#wine_data_binned = frequency_binning(wine_data, 'pH', 5)
#print("Value counts")
#print(wine_data_binned['pH'].value_counts())
#print("\nBucket Min Max Size")
#for i in wine_data_binned['pH'].unique():
#    max = wine_data[wine_data_binned['pH']==i]['pH'].max()
#    min = wine_data[wine_data_binned['pH']==i]['pH'].min()  
#    print(str(i) + " " + str(min) + " " + str(max) + " " + str(round(max-min,2)))


############### Mushroom Dataset Encoding ###############
# list of headinds that need to be one hot encoded
prefixList = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'gill-attachment', 'gill-spacing', 'gill-size', 
              'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 
              'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'ring-type', 
              'spore-print-color', 'population', 'habitat']

# veil-type column is not taken forward because each row has the same values making it useless for the models
mushroomsNew = mushrooms['class'] # the first enoded column (poisonus = 1, edible = 2)
# loop to convert onehot for all relevent 
for pref in prefixList:
    tempEncode = pd.get_dummies(mushrooms[pref], prefix=pref)
    mushroomsNew = pd.concat([mushroomsNew, tempEncode], axis=1)
print(mushroomsNew.head())

balancedDataWine = balancedDataWine.dropna()
# saves new datasets
mushroomsNew.to_csv('newMushroom.csv', index=False)
balancedDataWine.to_csv('newWine.csv', index=False)


class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64
type                     0
fixed-acidity           10
volatile-acidity         8
citric-acid              3
residual-sugar           2
chlorides                2
free-sulfur-dioxide      0
total-sulfur-dioxide     0
density                  0
pH                       9
sulphates                4
